In [16]:
# RAG-based Chatbot with PDF, CSV, Excel Support
# For Google Colab - Free and Open Source Implementation

# Install required packages
!pip install pinecone-client langchain langchain-community sentence-transformers
!pip install PyPDF2 pandas openpyxl faiss-cpu
!pip install google-generativeai langchain-google-genai
!pip install streamlit gradio

import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# File processing imports
import PyPDF2
from io import BytesIO
import json

# Vector database and embeddings
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# LLM imports
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

# UI
import gradio as gr

class MultiFormatRAGChatbot:
    def __init__(self, pinecone_api_key: str, gemini_api_key: str):
        """
        Initialize the RAG chatbot with Pinecone and Gemini API keys
        """
        self.pinecone_api_key = pinecone_api_key
        self.gemini_api_key = gemini_api_key

        # Initialize Pinecone client
        self.pc = Pinecone(api_key=pinecone_api_key)

        # Your Pinecone index configuration
        self.index_name = "rag-chatbot-csv"
        self.index_host = "https://rag-chatbot-csv-xp61bOh.svc.aped-4627-b74a.pinecone.io"

        # Initialize embeddings model (matching your Pinecone dimensions: 1024)
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

        # Initialize LLM
        genai.configure(api_key=gemini_api_key)
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            google_api_key=gemini_api_key,
            temperature=0.3,
            convert_system_message_to_human=True
        )

        # Text splitter for chunking documents
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

        # Connect to Pinecone index
        self.setup_pinecone_index()

        print("✅ RAG Chatbot initialized successfully!")

    def setup_pinecone_index(self):
        """Setup Pinecone index connection"""
        try:
            # Check if index exists
            existing_indexes = self.pc.list_indexes()
            index_names = [index.name for index in existing_indexes.indexes]

            if self.index_name not in index_names:
                # Create index if it doesn't exist
                self.pc.create_index(
                    name=self.index_name,
                    dimension=1024,  # Matching your configuration
                    metric='cosine',
                    spec=ServerlessSpec(
                        cloud='aws',
                        region='us-east-1'
                    )
                )
                print(f"✅ Created new Pinecone index: {self.index_name}")

            self.index = self.pc.Index(self.index_name)
            print(f"✅ Connected to Pinecone index: {self.index_name}")

        except Exception as e:
            print(f"❌ Error setting up Pinecone index: {str(e)}")
            raise

    def extract_text_from_pdf(self, pdf_file) -> str:
        """Extract text from PDF file"""
        try:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            return text
        except Exception as e:
            print(f"❌ Error extracting text from PDF: {str(e)}")
            return ""

    def process_csv_file(self, csv_file) -> str:
        """Process CSV file and convert to text"""
        try:
            df = pd.read_csv(csv_file)

            # Create a comprehensive text representation
            text_content = f"CSV Dataset Summary:\n"
            text_content += f"Total rows: {len(df)}\n"
            text_content += f"Total columns: {len(df.columns)}\n"
            text_content += f"Columns: {', '.join(df.columns)}\n\n"

            # Add column information
            text_content += "Column Information:\n"
            for col in df.columns:
                text_content += f"- {col}: {df[col].dtype}\n"

            # Add sample data
            text_content += f"\nSample Data (first 5 rows):\n"
            text_content += df.head().to_string()

            # Add statistical summary for numeric columns
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) > 0:
                text_content += f"\n\nStatistical Summary for Numeric Columns:\n"
                text_content += df[numeric_cols].describe().to_string()

            # Add all data in a structured format
            text_content += f"\n\nComplete Dataset:\n"
            for idx, row in df.iterrows():
                row_text = f"Row {idx + 1}: "
                for col in df.columns:
                    row_text += f"{col}: {row[col]}, "
                text_content += row_text.rstrip(", ") + "\n"

            return text_content

        except Exception as e:
            print(f"❌ Error processing CSV file: {str(e)}")
            return ""

    def process_excel_file(self, excel_file) -> str:
        """Process Excel file and convert to text"""
        try:
            # Read all sheets
            xls = pd.ExcelFile(excel_file)
            text_content = f"Excel File Summary:\n"
            text_content += f"Total sheets: {len(xls.sheet_names)}\n"
            text_content += f"Sheet names: {', '.join(xls.sheet_names)}\n\n"

            for sheet_name in xls.sheet_names:
                df = pd.read_excel(excel_file, sheet_name=sheet_name)

                text_content += f"\n{'='*50}\n"
                text_content += f"Sheet: {sheet_name}\n"
                text_content += f"{'='*50}\n"

                # Add sheet summary
                text_content += f"Rows: {len(df)}, Columns: {len(df.columns)}\n"
                text_content += f"Columns: {', '.join(df.columns)}\n\n"

                # Add sample data
                text_content += f"Sample Data (first 5 rows):\n"
                text_content += df.head().to_string()

                # Add all data
                text_content += f"\n\nComplete Data:\n"
                for idx, row in df.iterrows():
                    row_text = f"Row {idx + 1}: "
                    for col in df.columns:
                        row_text += f"{col}: {row[col]}, "
                    text_content += row_text.rstrip(", ") + "\n"

                text_content += "\n"

            return text_content

        except Exception as e:
            print(f"❌ Error processing Excel file: {str(e)}")
            return ""

    def create_embeddings(self, text: str) -> List[float]:
        """Create embeddings for text using SentenceTransformer"""
        try:
            embeddings = self.embedding_model.encode(text)
            # Pad or truncate to match Pinecone dimension (1024)
            if len(embeddings) < 1024:
                embeddings = np.pad(embeddings, (0, 1024 - len(embeddings)), 'constant')
            elif len(embeddings) > 1024:
                embeddings = embeddings[:1024]

            return embeddings.tolist()
        except Exception as e:
            print(f"❌ Error creating embeddings: {str(e)}")
            return []

    def add_documents_to_index(self, documents: List[Document], file_type: str, filename: str):
        """Add documents to Pinecone index"""
        try:
            vectors = []
            for i, doc in enumerate(documents):
                embedding = self.create_embeddings(doc.page_content)
                if embedding:
                    vectors.append({
                        'id': f"{filename}_{file_type}_{i}",
                        'values': embedding,
                        'metadata': {
                            'text': doc.page_content,
                            'file_type': file_type,
                            'filename': filename,
                            'chunk_id': i
                        }
                    })

            # Upsert vectors to Pinecone
            if vectors:
                self.index.upsert(vectors=vectors)
                print(f"✅ Added {len(vectors)} chunks to Pinecone index")

        except Exception as e:
            print(f"❌ Error adding documents to index: {str(e)}")

    def process_and_store_file(self, file_path: str, file_type: str):
        """Process file and store in vector database"""
        try:
            filename = os.path.basename(file_path)

            # Extract text based on file type
            if file_type == 'pdf':
                text = self.extract_text_from_pdf(file_path)
            elif file_type == 'csv':
                text = self.process_csv_file(file_path)
            elif file_type in ['xlsx', 'xls']:
                text = self.process_excel_file(file_path)
            else:
                raise ValueError(f"Unsupported file type: {file_type}")

            if not text:
                return f"❌ Failed to extract text from {filename}"

            # Split text into chunks
            documents = self.text_splitter.create_documents([text])

            # Add to vector database
            self.add_documents_to_index(documents, file_type, filename)

            return f"✅ Successfully processed and stored {filename} ({len(documents)} chunks)"

        except Exception as e:
            return f"❌ Error processing file: {str(e)}"

    def search_similar_documents(self, query: str, top_k: int = 5) -> List[Dict]:
        """Search for similar documents in Pinecone"""
        try:
            # Create query embedding
            query_embedding = self.create_embeddings(query)

            # Search in Pinecone
            search_results = self.index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True
            )

            # Format results
            results = []
            for match in search_results['matches']:
                results.append({
                    'text': match['metadata']['text'],
                    'score': match['score'],
                    'file_type': match['metadata']['file_type'],
                    'filename': match['metadata']['filename']
                })

            return results

        except Exception as e:
            print(f"❌ Error searching documents: {str(e)}")
            return []

    def generate_response(self, query: str, context_docs: List[Dict]) -> str:
        """Generate response using Gemini with context"""
        try:
            # Prepare context
            context = "\n\n".join([doc['text'] for doc in context_docs])

            # Create prompt
            prompt = f"""
            You are a helpful AI assistant that answers questions based on the provided context from uploaded documents.

            Context from documents:
            {context}

            User Question: {query}

            Instructions:
            1. Answer the question based on the provided context
            2. If the context doesn't contain enough information, say so
            3. Be specific and cite relevant information from the context
            4. If working with CSV/Excel data, provide specific data points when relevant

            Answer:
            """

            # Generate response
            response = self.llm.predict(prompt)
            return response

        except Exception as e:
            return f"❌ Error generating response: {str(e)}"

    def chat(self, query: str) -> str:
        """Main chat function"""
        try:
            # Search for relevant documents
            relevant_docs = self.search_similar_documents(query, top_k=5)

            if not relevant_docs:
                return "❌ No relevant documents found. Please upload some files first."

            # Generate response
            response = self.generate_response(query, relevant_docs)

            # Add source information
            sources = list(set([doc['filename'] for doc in relevant_docs]))
            response += f"\n\n📄 Sources: {', '.join(sources)}"

            return response

        except Exception as e:
            return f"❌ Error in chat: {str(e)}"

# Gradio Interface
def create_gradio_interface():
    """Create Gradio interface for the chatbot"""

    # Initialize chatbot (you'll need to provide your API keys)
    chatbot = None

    def initialize_chatbot(pinecone_key, gemini_key):
        global chatbot
        try:
            chatbot = MultiFormatRAGChatbot(pinecone_key, gemini_key)
            return "✅ Chatbot initialized successfully!"
        except Exception as e:
            return f"❌ Error initializing chatbot: {str(e)}"

    def upload_and_process_file(file):
        global chatbot
        if chatbot is None:
            return "❌ Please initialize the chatbot first with your API keys."

        if file is None:
            return "❌ Please upload a file."

        try:
            # Determine file type
            filename = file.name
            if filename.endswith('.pdf'):
                file_type = 'pdf'
            elif filename.endswith('.csv'):
                file_type = 'csv'
            elif filename.endswith(('.xlsx', '.xls')):
                file_type = 'xlsx'
            else:
                return "❌ Unsupported file type. Please upload PDF, CSV, or Excel files."

            # Process file
            result = chatbot.process_and_store_file(file.name, file_type)
            return result

        except Exception as e:
            return f"❌ Error processing file: {str(e)}"

    def chat_with_bot(message, history):
        global chatbot
        if chatbot is None:
            return "❌ Please initialize the chatbot first with your API keys."

        response = chatbot.chat(message)
        return response

    # Create Gradio interface
    with gr.Blocks(title="RAG Chatbot - Multi-Format File Support") as demo:
        gr.Markdown("# 🤖 RAG Chatbot with PDF, CSV, Excel Support")
        gr.Markdown("Upload your documents and chat with them using AI!")

        with gr.Tab("Setup"):
            gr.Markdown("## 🔧 Initialize Chatbot")
            with gr.Row():
                pinecone_key = gr.Textbox(
                    label="Pinecone API Key",
                    type="password",
                    placeholder="Enter your Pinecone API key"
                )
                gemini_key = gr.Textbox(
                    label="Google Gemini API Key",
                    type="password",
                    placeholder="Enter your Google Gemini API key"
                )

            init_btn = gr.Button("Initialize Chatbot", variant="primary")
            init_output = gr.Textbox(label="Initialization Status", interactive=False)

            init_btn.click(
                initialize_chatbot,
                inputs=[pinecone_key, gemini_key],
                outputs=init_output
            )

        with gr.Tab("Upload Files"):
            gr.Markdown("## 📁 Upload Documents")
            file_upload = gr.File(
                label="Upload PDF, CSV, or Excel files",
                file_types=[".pdf", ".csv", ".xlsx", ".xls"]
            )
            upload_btn = gr.Button("Process File", variant="primary")
            upload_output = gr.Textbox(label="Processing Status", interactive=False)

            upload_btn.click(
                upload_and_process_file,
                inputs=file_upload,
                outputs=upload_output
            )

        with gr.Tab("Chat"):
            gr.Markdown("## 💬 Chat with Your Documents")
            chatbot_interface = gr.Chatbot(height=400)
            msg = gr.Textbox(
                label="Your Message",
                placeholder="Ask questions about your uploaded documents..."
            )

            def respond(message, chat_history):
                bot_message = chat_with_bot(message, chat_history)
                chat_history.append((message, bot_message))
                return "", chat_history

            msg.submit(respond, [msg, chatbot_interface], [msg, chatbot_interface])

            gr.Examples(
                examples=[
                    "What is the main topic of the uploaded document?",
                    "Can you summarize the key findings?",
                    "What data is available in the CSV file?",
                    "Show me the statistics from the Excel sheet",
                    "What are the column names in the dataset?"
                ],
                inputs=msg
            )

    return demo

# Usage Instructions
print("""
🚀 RAG Chatbot Setup Instructions:

1. **Get API Keys:**
   - Pinecone: https://www.pinecone.io/ (Free tier available)
   - Google Gemini: https://ai.google.dev/ (Free tier available)

2. **Run the Gradio Interface:**
   ```python
   demo = create_gradio_interface()
   demo.launch(share=True, debug=True)
   ```

3. **Initialize the chatbot with your API keys**

4. **Upload your PDF, CSV, or Excel files**

5. **Start chatting with your documents!**

📝 Features:
- ✅ PDF text extraction
- ✅ CSV data analysis
- ✅ Excel multi-sheet support
- ✅ Vector similarity search
- ✅ Context-aware responses
- ✅ Source attribution
- ✅ Free and open-source
""")

# Launch the interface
if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(share=True, debug=True)


🚀 RAG Chatbot Setup Instructions:

1. **Get API Keys:**
   - Pinecone: https://www.pinecone.io/ (Free tier available)
   - Google Gemini: https://ai.google.dev/ (Free tier available)

2. **Run the Gradio Interface:**
   ```python
   demo = create_gradio_interface()
   demo.launch(share=True, debug=True)
   ```

3. **Initialize the chatbot with your API keys**

4. **Upload your PDF, CSV, or Excel files**

5. **Start chatting with your documents!**

📝 Features:
- ✅ PDF text extraction
- ✅ CSV data analysis
- ✅ Excel multi-sheet support
- ✅ Vector similarity search
- ✅ Context-aware responses
- ✅ Source attribution
- ✅ Free and open-source

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1d049385c582ab59e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy

✅ Connected to Pinecone index: rag-chatbot-csv
✅ RAG Chatbot initialized successfully!
✅ Added 96 chunks to Pinecone index
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1d049385c582ab59e0.gradio.live
